Пример № 1. SciPy (scipy.optimize.linprog)

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [38]:
from scipy.optimize import linprog, LinearConstraint, milp
import numpy as np


In [30]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров


In [31]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив


In [32]:
linprog(c=c, A_ub=A, b_ub=b)


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

Пример № 2. CVXPY

Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку CVXPY.


In [33]:
import cvxpy


In [34]:
x = cvxpy.Variable(shape=n, integer = True) #С помощью CVXPY создадим переменную-массив. Укажем его размерность, а также условие, что все числа в массиве должны быть целыми


In [35]:
#Далее зададим ограничения, используя матричное умножение
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))


In [36]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])


In [37]:
problem.solve()


SolverError: Solver 'SCIPY' failed. Try another solver, or solve with verbose=True for more information.

In [13]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value


array([-0., -0., -0.,  7., -0.,  0.])

In [14]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value


array([1., 0., 0., 1., 0., 1.])

Пример № 3. PuLP

В нашей каршеринговой компании две модели автомобилей: модель A и модель B. Автомобиль A даёт прибыль в размере 20 тысяч в месяц, а автомобиль B — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа A, и 5 дней — для каждого автомобиля типа B.
Заводу требуется 3 дня, чтобы изготовить модель A, и 6 дней, чтобы изготовить модель B.
Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль A, и 7 дней —  автомобиль B.
Каждый специалист может работать суммарно 30 дней.

In [15]:
from pulp import *


In [16]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))


Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


c:\Users\lepni\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:
![Alt text](C:\IDE\MATH+ML\ML_6\MATHML_md6_6_1_1.png)

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом (желательно программным).

В качестве ответа введите минимальную суммарную стоимость поставки. Ответ округлите до целого числа.

In [27]:
WH_prob = LpProblem('Стоимость логистики', LpMinimize)
w1c1 = LpVariable('роут 1', lowBound=0 , cat=LpInteger)
w1c2 = LpVariable('роут 2', lowBound=0 , cat=LpInteger)
w1c3 = LpVariable('роут 3', lowBound=0 , cat=LpInteger)
w2c1 = LpVariable('роут 4', lowBound=0 , cat=LpInteger)
w2c2 = LpVariable('роут 5', lowBound=0 , cat=LpInteger)
w2c3 = LpVariable('роут 6', lowBound=0 , cat=LpInteger)
#Целевая функция
WH_prob += 2*w1c1+5*w1c2+3*w1c3+7*w2c1+7*w2c2+6*w2c3
#Ограничения
WH_prob += w1c1 + w1c2 + w1c3 == 180
WH_prob += w2c1 + w2c2 + w2c3 == 220
WH_prob += w1c1 + w2c1 <= 110
WH_prob += w1c2 + w2c2 <= 150
WH_prob += w1c3 + w2c3 <= 140

WH_prob.solve()

for v in WH_prob.variables():
    print (v.name, "=", v.varValue)
  
print ("objective = %s$" % value(WH_prob.objective))


роут_1 = 110.0
роут_2 = 0.0
роут_3 = 70.0
роут_4 = 0.0
роут_5 = 150.0
роут_6 = 70.0
objective = 1900.0$
